In [106]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##dgl.DGLGraph.apply_nodes

In [107]:
import dgl
import torch

In [108]:
g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['h'] = torch.ones(5, 2)
g.apply_nodes(lambda nodes: {'x' : nodes.data['h'] * 2})
g.ndata['x']

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

In [109]:
g = dgl.heterograph({('user', 'follows', 'user'): ([0, 1], [1, 2])})
g.nodes['user'].data['h'] = torch.ones(3, 5)
g.apply_nodes(lambda nodes: {'h': nodes.data['h'] * 2}, ntype='user')
g.nodes['user'].data['h']

tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])

##dgl.DGLGraph.apply_edges

In [110]:
g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['h'] = torch.ones(5, 2)
g.apply_edges(lambda edges: {'x' : edges.src['h'] + edges.dst['h']})
g.edata['x']

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

In [111]:
import dgl.function as fn
g.apply_edges(fn.u_add_v('h', 'h', 'x'))
g.edata['x']

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

In [112]:
g = dgl.heterograph({('user', 'plays', 'game'): ([0, 1, 1, 2], [0, 0, 2, 1])})
g.edges[('user', 'plays', 'game')].data['h'] = torch.ones(4, 5)
g.apply_edges(lambda edges: {'h': edges.data['h'] * 2})
g.edges[('user', 'plays', 'game')].data['h']

tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])

##dgl.DGLGraph.send_and_recv

In [113]:
import dgl.function as fn
g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['x'] = torch.ones(5, 2)
# Specify edges using (Tensor, Tensor).
g.send_and_recv(([1, 2], [2, 3]), fn.copy_u('x', 'm'), fn.sum('m', 'h'))
g.ndata['h']
# Specify edges using IDs.
g.send_and_recv([0, 2, 3], fn.copy_u('x', 'm'), fn.sum('m', 'h'))
g.ndata['h']

tensor([[0., 0.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [114]:
g = dgl.heterograph({
    ('user', 'follows', 'user'): ([0, 1], [1, 2]),
    ('user', 'plays', 'game'): ([0, 1, 1, 2], [0, 0, 1, 1])
})
g.nodes['user'].data['h'] = torch.tensor([[0.], [1.], [2.]])
g.send_and_recv(g['follows'].edges(), fn.copy_u('h', 'm'),
                fn.sum('m', 'h'), etype='follows')
g.nodes['user'].data['h']

tensor([[0.],
        [0.],
        [1.]])

In [115]:
import torch as th
g = dgl.graph(([0, 1], [1, 2]))
g.ndata['x'] = th.tensor([[1.], [2.], [3.]])

In [116]:
# Define the function for sending node features as messages.
def send_source(edges):
    return {'m': edges.src['x']}
# Sum the messages received and use this to replace the original node feature.
def simple_reduce(nodes):
    return {'x': nodes.mailbox['m'].sum(1)}

In [117]:
g.send_and_recv(g.edges())
g.ndata['x']

TypeError: ignored

##dgl.DGLGraph.pull

In [118]:
g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['x'] = torch.ones(5, 2)
g.pull([0, 3, 4], fn.copy_u('x', 'm'), fn.sum('m', 'h'))
g.ndata['h']

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 1.],
        [1., 1.]])

In [119]:
g = dgl.heterograph({
    ('user', 'follows', 'user'): ([0, 1], [1, 2]),
    ('user', 'plays', 'game'): ([0, 2], [0, 1])
})
g.nodes['user'].data['h'] = torch.tensor([[0.], [1.], [2.]])

In [120]:
g['follows'].pull(2, fn.copy_u('h', 'm'), fn.sum('m', 'h'), etype='follows')
g.nodes['user'].data['h']

tensor([[0.],
        [1.],
        [1.]])

##dgl.DGLGraph.push

In [121]:
g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['x'] = torch.ones(5, 2)
g.push([0, 1], fn.copy_u('x', 'm'), fn.sum('m', 'h'))
g.ndata['h']

tensor([[0., 0.],
        [1., 1.],
        [1., 1.],
        [0., 0.],
        [0., 0.]])

In [122]:
g = dgl.heterograph({('user', 'follows', 'user'): ([0, 0], [1, 2])})
g.nodes['user'].data['h'] = torch.tensor([[0.], [1.], [2.]])

In [123]:
g['follows'].push(0, fn.copy_u('h', 'm'), fn.sum('m', 'h'), etype='follows')
g.nodes['user'].data['h']

tensor([[0.],
        [0.],
        [0.]])

##dgl.DGLGraph.update_all

In [124]:
g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['x'] = torch.ones(5, 2)
g.update_all(fn.copy_u('x', 'm'), fn.sum('m', 'h'))
g.ndata['h']

tensor([[0., 0.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [125]:
g = dgl.heterograph({('user', 'follows', 'user'): ([0, 1, 2], [1, 2, 2])})

In [126]:
g.nodes['user'].data['h'] = torch.tensor([[0.], [1.], [2.]])
g['follows'].update_all(fn.copy_u('h', 'm'), fn.sum('m', 'h'), etype='follows')
g.nodes['user'].data['h']

tensor([[0.],
        [0.],
        [3.]])

In [127]:
g = dgl.heterograph({
    ('user', 'follows', 'user'): ([0, 1], [1, 1]),
    ('game', 'attracts', 'user'): ([0], [1])
})

In [128]:
g.nodes['user'].data['h'] = torch.tensor([[1.], [2.]])
g.nodes['game'].data['h'] = torch.tensor([[1.]])
g.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'h'))
g.nodes['user'].data['h']

tensor([[0.],
        [4.]])

##dgl.function.copy_u

In [129]:
message_func = dgl.function.copy_u('h', 'm')

In [130]:
def message_func(edges):
    return {'m': edges.src['h']}

##dgl.function.copy_e

In [131]:
message_func = dgl.function.copy_e('h', 'm')

In [132]:
def message_func(edges):
    return {'m': edges.data['h']}

##dgl.function.u_add_v


In [133]:
message_func = dgl.function.u_add_v('h', 'h', 'm')

##dgl.function.u_mul_v

In [134]:
message_func = dgl.function.u_mul_v('h', 'h', 'm')

##dgl.function.v_add_e

In [135]:
message_func = dgl.function.v_add_e('h', 'h', 'm')

##dgl.function.v_mul_e

In [136]:
message_func = dgl.function.v_mul_e('h', 'h', 'm')

##dgl.function.v_add_u

In [137]:
message_func = dgl.function.v_add_u('h', 'h', 'm')

##import dgl
message_func = dgl.function.v_add_u('h', 'h', 'm')

In [138]:
message_func = dgl.function.v_mul_u('h', 'h', 'm')

##dgl.function.u_add_e

In [139]:
message_func = dgl.function.u_add_e('h', 'h', 'm')

##dgl.function.u_mul_e

In [140]:
message_func = dgl.function.u_mul_e('h', 'h', 'm')

##1

1. ``g.apply_nodes'' function:
    - In this piece of code, a graph `g' with 5 nodes is created.
    - Each node is assigned a feature vector of size 2 using `g.ndata['h'] = torch.ones(5, 2)`.
    - The `apply_nodes` function is called on the graph, which applies a user-defined function to each node and updates the nodes' data accordingly.
    - The lambda function `lambda nodes: {'x': nodes.data['h'] * 2}` is applied to each node, which doubles the existing properties of the node, and the property 'x' of each node to makes day
    - Updated node data output, `g.ndata['x']` which is a tensor of dimensions (5, 2) padded with 2.

2. ``g.apply_edges'' function:
    - A similar process is repeated for the edges in the graph.
    - The lambda function `lambda edges: {'x': edges.src['h'] + edges.dst['h']}` is applied to each edge, combining the properties of the source and destination nodes. And the 'x' attribute updates each edge.
    - Updated edge data output, `g.edata['x']` which is a tensor of size (4, 2) padded with 2.

3. ``g.send_and_recv'' function:
    - ``send_and_recv'' function is used to send messages through edges and perform reduction operations on nodes.
    - The first call to `g.send_and_recv([1, 2], fn.copy_u('x', 'm'), fn.sum('m', 'h'))` returns messages from nodes 1 and 2 It sends to its neighbor nodes using the 'x' attribute and the messages are aggregated at the receiving nodes to update the 'h' attribute of each node.
    - The second call `g.send_and_recv([0, 2, 3], fn.copy_u('x', 'm'), fn.sum('m', 'h'))` does a similar operation for nodes 0 , 2 and 3 do.
    - Updated node data output, `g.ndata['h']` which is a tensor of size (5, 2) padded with 2.
    4. `g.pull`:
- The purpose of the `pull` function is to update the node features by collecting messages from neighboring nodes.
- For example, calling `g.pull([0, 3, 4], fn.copy_u('x', 'm'), fn.sum('m', 'h'))` retrieves messages from nodes 0, 3, and 4 using the 'x' feature, and then these messages are summed at the receiving nodes to update the 'h' feature of each node.
- The result is the updated node data, `g.ndata['h']`, which is a tensor of size (5, 2) filled with the value 2.

5. `g.push`:
- The purpose of the `push` function is to send messages from nodes to their neighboring nodes and perform reduction operations.
- For instance, calling `g.push([0, 1], fn.copy_u('x', 'm'), fn.sum('m', 'h'))` sends messages from nodes 0 and 1 using the 'x' feature, and then these messages are summed at the receiving nodes to update the 'h' feature of each node.
- The output is the updated node data, `g.ndata['h']`, which is a tensor of size (5, 2) filled with the value 2.

6. `g.update_all`:
- The `update_all` function combines the functionalities of `send_and_recv` and `pull`, allowing messages to be sent along edges and reduction operations to be performed on nodes simultaneously.
- For instance, calling `g.update_all(fn.copy_u('x', 'm'), fn.sum('m', 'h'))` sends messages from each node to its neighboring nodes using the 'x' feature, and then these messages are summed at the receiving nodes to update the 'h' feature of each node.
- The result is the updated node data, `g.ndata['h']`, which is a tensor of size (5, 2) filled with the value 2.

In [154]:
!pip install dgl

import dgl
import torch

"""## dgl.DGLGraph.apply_nodes"""

g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['h'] = torch.ones(5, 2)
g.apply_nodes(lambda nodes: {'x': nodes.data['h'] * 2})
print(g.ndata['x'])

g = dgl.heterograph({('user', 'follows', 'user'): ([0, 1], [1, 2])})
g.nodes['user'].data['h'] = torch.ones(3, 5)
g.apply_nodes(lambda nodes: {'h': nodes.data['h'] * 2}, ntype='user')
print(g.nodes['user'].data['h'])

"""## dgl.DGLGraph.apply_edges"""

g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['h'] = torch.ones(5, 2)
g.apply_edges(lambda edges: {'x': edges.src['h'] + edges.dst['h']})
print(g.edata['x'])

import dgl.function as fn
g.apply_edges(fn.u_add_v('h', 'h', 'x'))
print(g.edata['x'])

g = dgl.heterograph({('user', 'plays', 'game'): ([0, 1, 1, 2], [0, 0, 2, 1])})
g.edges[('user', 'plays', 'game')].data['h'] = torch.ones(4, 5)
g.apply_edges(lambda edges: {'h': edges.data['h'] * 2})
print(g.edges[('user', 'plays', 'game')].data['h'])

"""## dgl.DGLGraph.send_and_recv"""

import dgl.function as fn
g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['x'] = torch.ones(5, 2)
g.send_and_recv([1, 2], fn.copy_u('x', 'm'), fn.sum('m', 'h'))
print(g.ndata['h'])

g.send_and_recv([0, 2, 3], fn.copy_u('x', 'm'), fn.sum('m', 'h'))
print(g.ndata['h'])

g = dgl.heterograph({
    ('user', 'follows', 'user'): ([0, 1], [1, 2]),
    ('user', 'plays', 'game'): ([0, 1, 1, 2], [0, 0, 1, 1])
})
g.nodes['user'].data['h'] = torch.tensor([[0.], [1.], [2.]])
g.send_and_recv(g['follows'].edges(), fn.copy_u('h', 'm'),
                fn.sum('m', 'h'), etype='follows')
print(g.nodes['user'].data['h'])

import torch as th
g = dgl.graph(([0, 1], [1, 2]))
g.ndata['x'] = th.tensor([[1.], [2.], [3.]])

def send_source(edges):
    return {'m': edges.src['x']}
def simple_reduce(nodes):
    return {'x': nodes.mailbox['m'].sum(1)}

g.send_and_recv(g.edges(), send_source, simple_reduce)
print(g.ndata['x'])

"""## dgl.DGLGraph.pull"""

g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['x'] = torch.ones(5, 2)
g.pull([0, 3, 4], fn.copy_u('x', 'm'), fn.sum('m', 'h'))
print(g.ndata['h'])

g = dgl.heterograph({
    ('user', 'follows', 'user'): ([0, 1], [1, 2]),
    ('user', 'plays', 'game'): ([0, 2], [0, 1])
})
g.nodes['user'].data['h'] = torch.tensor([[0.], [1.], [2.]])

g['follows'].pull(2, fn.copy_u('h', 'm'), fn.sum('m', 'h'), etype='follows')
print(g.nodes['user'].data['h'])

"""## dgl.DGLGraph.push"""

g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['x'] = torch.ones(5, 2)
g.push([0, 1], fn.copy_u('x', 'm'), fn.sum('m', 'h'))
print(g.ndata['h'])

g = dgl.heterograph({('user', 'follows', 'user'): ([0, 0], [1, 2])})
g.nodes['user'].data['h'] = torch.tensor([[0.], [1.], [2.]])

g['follows'].push(0, fn.copy_u('h', 'm'), fn.sum('m', 'h'), etype='follows')
print(g.nodes['user'].data['h'])

"""## dgl.DGLGraph.update_all"""

g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))
g.ndata['x'] = torch.ones(5, 2)
g.update_all(fn.copy_u('x', 'm'), fn.sum('m', 'h'))
print(g.ndata['h'])

g = dgl.heterograph({('user', 'follows', 'user'): ([0, 1, 2], [1, 2, 2])})

g.nodes['user'].data['h'] = torch.tensor([[0.], [1.], [2.]])
g['follows'].update_all(fn.copy_u('h', 'm'), fn.sum('m', 'h'), etype='follows')
print(g.nodes['user'].data['h'])

g = dgl.heterograph({
    ('user', 'follows', 'user'): ([0, 1], [1, 1]),
    ('game', 'attracts', 'user'): ([0], [1])
})

g.nodes['user'].data['h'] = torch.tensor([[1.], [2.]])
g.nodes['game'].data['h'] = torch.tensor([[1.]])
g.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'h'))
print(g.nodes['user'].data['h'])


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])
tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])
tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])
tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])
tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])
tensor([[0., 0.],
        [0., 0.],
        [1., 1.],
        [1., 1.],
        [0., 0.]])
tensor([[0., 0.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[0.],
        [0.],
        [1.]])
tensor([[1.],
        [1.],
        [2.]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 1.],
        [1., 1.]])
tensor([[0.],
        [1.],
        [1.]])
tensor([[0., 0.],
        [1., 1.],
        [1., 1

##2

1. Graph Creation:
   - This code snippet creates a directed graph named `g` using the function `dgl.graph()`.
   - The graph consists of 5 nodes and 4 edges, which are defined by the lists `[0, 1, 2, 3]` and `[1, 2, 3, 4]` respectively.

2. Node and Edge Initialization:
   - The code initializes node features by assigning values to the attribute `g.ndata['h']`, which is a torch tensor containing `[[1.], [2.], [3.], [4.], [5.]]`.
   - Edge features are also initialized in a similar manner using the attribute `g.edata['h']`, with values `[[1.], [2.], [3.], [4.]]`.

3. Copying Node Features to Neighboring Nodes:
   - To copy node feature `'h'` to message data `'m'`, the function `dgl.function.copy_u('h', 'm')` is utilized.
   - The messages `'m'` are then summed at the receiver nodes and the updated feature `'h'` is stored there, using the function `dgl.function.sum('m', 'h')`.
   - The `send_and_recv` function is applied to the graph `g` with all edges as arguments, executing the copy and sum operations.
   - The node features `'h'` are printed after this operation.

4. Copying Edge Features to Destination Nodes:
   - Similar to the previous step, the function `dgl.function.copy_e('h', 'm')` is used to copy the edge feature `'h'` to message data `'m'`.
   - The messages `'m'` are summed at the receiver nodes and the updated feature `'h'` is stored there using the function `dgl.function.sum('m', 'h')`.
   - The `send_and_recv` function is called again, but this time copying and summing edge features.
   - The node features `'h'` are printed after this operation.

5. Node and Edge Feature Operations:
   - This section involves performing various operations using different functions from the `dgl.function` module.
   - These operations include addition and multiplication of node features, addition of edge features to node features, and addition of edge features to source node features.
   - The `send_and_recv` function is called for each operation, updating the node features `'h'`.
   - The node features `'h'` are printed after each operation.

The output of the code displays the updated node features `'h'` after each operation. It serves as a demonstration of how messages can be sent and received between nodes in a graph, and how node and edge features can be manipulated using the DGL library.

In [145]:
import dgl
import torch

g = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))

# Initialize node data
g.ndata['h'] = torch.tensor([[1.], [2.], [3.], [4.], [5.]])

# Initialize edge data
g.edata['h'] = torch.tensor([[1.], [2.], [3.], [4.]])

# Copy node feature to neighboring nodes
g.send_and_recv(g.edges(), dgl.function.copy_u('h', 'm'), dgl.function.sum('m', 'h'))
print(g.ndata['h'])

# Copy edge feature to destination nodes
g.send_and_recv(g.edges(), dgl.function.copy_e('h', 'm'), dgl.function.sum('m', 'h'))
print(g.ndata['h'])

# Add source and destination node features
g.send_and_recv(g.edges(), dgl.function.u_add_v('h', 'h', 'm'), dgl.function.sum('m', 'h'))
print(g.ndata['h'])

# Multiply source and destination node features
g.send_and_recv(g.edges(), dgl.function.u_mul_v('h', 'h', 'm'), dgl.function.sum('m', 'h'))
print(g.ndata['h'])

# Add edge feature to destination node feature
g.send_and_recv(g.edges(), dgl.function.v_add_e('h', 'h', 'm'), dgl.function.sum('m', 'h'))
print(g.ndata['h'])

# Multiply edge feature to destination node feature
g.send_and_recv(g.edges(), dgl.function.v_mul_e('h', 'h', 'm'), dgl.function.sum('m', 'h'))
print(g.ndata['h'])

# Add node feature to destination node feature
g.send_and_recv(g.edges(), dgl.function.v_add_u('h', 'h', 'm'), dgl.function.sum('m', 'h'))
print(g.ndata['h'])

# Multiply node feature to destination node feature
g.send_and_recv(g.edges(), dgl.function.v_mul_u('h', 'h', 'm'), dgl.function.sum('m', 'h'))
print(g.ndata['h'])

# Add edge feature to source node feature
g.send_and_recv(g.edges(), dgl.function.u_add_e('h', 'h', 'm'), dgl.function.sum('m', 'h'))
print(g.ndata['h'])

# Multiply edge feature to source node feature
g.send_and_recv(g.edges(), dgl.function.u_mul_e('h', 'h', 'm'), dgl.function.sum('m', 'h'))
print(g.ndata['h'])


tensor([[1.],
        [1.],
        [2.],
        [3.],
        [4.]])
tensor([[1.],
        [1.],
        [2.],
        [3.],
        [4.]])
tensor([[1.],
        [2.],
        [3.],
        [5.],
        [7.]])
tensor([[ 1.],
        [ 2.],
        [ 6.],
        [15.],
        [35.]])
tensor([[ 1.],
        [ 3.],
        [ 8.],
        [18.],
        [39.]])
tensor([[  1.],
        [  3.],
        [ 16.],
        [ 54.],
        [156.]])
tensor([[  1.],
        [  4.],
        [ 19.],
        [ 70.],
        [210.]])
tensor([[1.0000e+00],
        [4.0000e+00],
        [7.6000e+01],
        [1.3300e+03],
        [1.4700e+04]])
tensor([[1.0000e+00],
        [2.0000e+00],
        [6.0000e+00],
        [7.9000e+01],
        [1.3340e+03]])
tensor([[  1.],
        [  1.],
        [  4.],
        [ 18.],
        [316.]])
